## Collecting protocol fees across Balancer core pools on all networks
Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [ ]:
# Query:
POOLS_SNAPSHOTS_QUERY = """
{{
  poolSnapshots(
    first: {first}
    skip: {skip}
    orderBy: timestamp
    orderDirection: desc
    block: {{ number: {block} }}
  ) {{
    pool {{
      address
      id
      symbol
    }}
    timestamp
    protocolFee
    swapFees
    swapVolume
    liquidity
  }}
}}
"""

In [ ]:

from datetime import datetime
from datetime import timedelta
from typing import Dict
from typing import List
from typing import Optional

from gql import Client
from gql import gql
from gql.transport.requests import RequestsHTTPTransport

BALANCER_GRAPH_URL = "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-arbitrum-v2"

# Start date and end date are the following: start date is always previous Thursday 00:00 UTC, end date is always this Thursday 00:00 UTC:
today = datetime.today()
# Calculate days until previous Thursday (0 = Monday, 3 = Thursday)
days_until_previous_thursday = (today.weekday() - 3) % 7
# Calculate start date by subtracting days_until_previous_thursday and setting time to 00:00
start_date = today - timedelta(days=days_until_previous_thursday, hours=today.hour, minutes=today.minute,
                               seconds=today.second, microseconds=today.microsecond)
# Calculate end date by adding 7 days to the start date
end_date = start_date + timedelta(days=7)

start_ts = int(start_date.timestamp())
end_ts = int(end_date.timestamp())


# Fetch all the data from the balancer subgraph
def make_gql_client(url: str) -> Optional[Client]:
    transport = RequestsHTTPTransport(url=url, retries=3)
    return Client(
        transport=transport, fetch_schema_from_transport=True, execute_timeout=60
    )


def get_balancer_pool_snapshots() -> Optional[List[Dict]]:
    client = make_gql_client(BALANCER_GRAPH_URL)
    all_pools = []
    limit = 100
    offset = 0
    while True:
        result = client.execute(
            gql(POOLS_SNAPSHOTS_QUERY.format(first=limit, skip=offset, start_ts=start_ts, end_ts=end_ts)))
        all_pools.extend(result['poolSnapshots'])
        offset += limit
        if len(result['poolSnapshots']) < limit - 1:
            break
    return all_pools


pool_snapshots = get_balancer_pool_snapshots()